JUNTO ACA TODA LA TRANSFORMACION a ver si la puedo hacer como una funcion:

In [1]:
#librerias a utilizar:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#para ver valores faltantes:
import missingno as msno
#para codificar:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder



In [2]:
#IMPORTANTE!!!: Le dejo el nombre TRAIN pero uso el TEST.CSV!, solo para no tener que cambiar en todos 
# lados 'train' por 'test'
df_train=pd.read_csv('hospitalizaciones_test.csv')

In [4]:
# ETL para df_test: solo borro donde aparece la columna 'Stay (in days)', que no esta en este otro df. 

#variable 'Insurance': 
le = preprocessing.LabelEncoder()
df_train['Insurance2']=le.fit_transform(df_train['Insurance'])

# variable 'Age':(notar q ya esta discretizada...)
le = preprocessing.LabelEncoder()
df_train['Age2']=le.fit_transform(df_train['Age'])

#lo hago manualmente: creo una nueva columna como una lista, y se la agrego a mi dataframe:
type_admission=[]
for row in df_train['Type of Admission']:
    if row == 'Trauma': type_admission.append(1)
    if row == 'Urgent': type_admission.append(2)
    if row == 'Emergency': type_admission.append(3)
#agrego la nueva columna, como esta lista. 
df_train['type_admission']=type_admission

#de la misma manera, codifico 'Severity of Illness':
severity_of_illnes=[]
for row in df_train['Severity of Illness']:
    if row == 'Minor': severity_of_illnes.append(0)
    if row == 'Moderate': severity_of_illnes.append(1)
    if row == 'Extreme': severity_of_illnes.append(2)

#agrego la nueva columna, como esta lista. 
df_train['severity_of_illnes']=severity_of_illnes

# variable target 'Stay (in days)': (
# agrego una nueva columna target:       
#primero, creo una lista y la poblo segun la condicion de consignas:
'''long_stay=[]
for row in df_train['Stay (in days)']:
    if row > 8: long_stay.append(1)
    else: long_stay.append(0)
#agrego la nueva columna, como esta lista. 
df_train['long_stay']=long_stay'''
 
gender = df_train[['gender']]
encoder = OneHotEncoder(handle_unknown = 'ignore')
gender2 = pd.DataFrame.sparse.from_spmatrix(encoder.fit_transform(gender), columns=encoder.get_feature_names_out(['gender'])) #defino columnas para no perder las denominaciones con .get_feature...!
#y se lo agrego a mi df:
df_train = df_train.join(gender2)

#aplico el mismo proced que para 'gender':
health_conditions = df_train[['health_conditions']] #...seria lo mismo con [] simples ?
encoder = OneHotEncoder(handle_unknown = 'ignore')
health_conditions2 = pd.DataFrame.sparse.from_spmatrix(encoder.fit_transform(health_conditions), columns=encoder.get_feature_names_out(['health_conditions'])) 
#y se lo agrego a mi df:
df_train = df_train.join(health_conditions2)
#checkeo q este ok en df_train.head(). OK!. DESVENTAJA: JUSTO ESTE FEATURE TIENE VARIOS VALORES, POR LO Q ME AGREGA DEMASIADAS COLLUMNAS, PERO igualmente, deberia tener 
# bastante peso en el modelo, ya que es un feature bastante relevante, o deberia serlo!!

#nuevamente el mismo procedimiento:
Department = df_train[['Department']] #...seria lo mismo con [] simples ?
encoder = OneHotEncoder(handle_unknown = 'ignore')
Department2 = pd.DataFrame.sparse.from_spmatrix(encoder.fit_transform(Department), columns=encoder.get_feature_names_out(['Department'])) 
#y se lo agrego a mi df:
df_train = df_train.join(Department2)

# mismo procedimiento:
doctor_name = df_train[['doctor_name']]
encoder = OneHotEncoder(handle_unknown = 'ignore')
doctor_name2 = pd.DataFrame.sparse.from_spmatrix(encoder.fit_transform(doctor_name), columns=encoder.get_feature_names_out(['doctor_name'])) 
#y se lo agrego a mi df:
df_train = df_train.join(doctor_name2)

#tiro las variables no numericas y las de corr_baja:
columnas=['Department','Ward_Facility_Code','patientid','Visitors with Patient','Admission_Deposit','doctor_name','Age','gender',
'Type of Admission', 'Severity of Illness', 'health_conditions','Insurance']  #,'Stay (in days)'] #las que seguro no me sirven.
df_train2 = df_train.drop(columnas, axis=1) 

#dropeo columnas que observé con correlacion muy baja o nula:
cols=['Available Extra Rooms in Hospital' , 'staff_available', 'Insurance2']
df_train4 = df_train2.drop(cols, axis=1)

In [7]:
#ahora checkeo a ver como quedó:
df_train4.head()
#ES ESTE EL Q TENGO Q TOMAR!

,Age2,type_admission,severity_of_illnes,gender_Female,gender_Male,gender_Other,health_conditions_Asthama,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,...,Department_surgery,doctor_name_Dr Isaac,doctor_name_Dr John,doctor_name_Dr Mark,doctor_name_Dr Nathan,doctor_name_Dr Olivia,doctor_name_Dr Sam,doctor_name_Dr Sarah,doctor_name_Dr Simon,doctor_name_Dr Sophia
0,2,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,5,1,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,1,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_train4.shape

(90000, 26)

In [9]:
df_TEST=df_train4.copy() # me hago una copia asi puedo chequear ene l otro notebook sin pisar nombres...
# ESTA OK EL NUMERO DE COLUMNAS. 


In [10]:
df_TEST.shape

(90000, 26)

In [11]:
#lo gauardo como csv, y ahora lo leo desde mi notebook 'EDA.ipynb'
df_TEST.to_csv('test_modificado.csv', index=False)

/var/folders/tf/dsk90f9d14gcg40_frclnb7r0000gn/T/ipykernel_20151/2616286075.py:1: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  df_TEST.to_csv('test_modificado.csv')
